# <span style="color:blue"> Alternative Clustering -- motifs</span> 
Nick Zolman

## <span style="color:red"> Wedges </span>
In the following algorithm, we compute a "wedge" version of the clustering coffecient. Where we have a wedge motif (which can be incoming or outgoing, e.g. $a \to c, b \to c$ or $ c \to a, c \to b$). Inspiration come from the intensity of motifs discussed in Onnela et al's paper: https://journals.aps.org/pre/pdf/10.1103/PhysRevE.71.065103

For a given node $a$ with weights $\{a_i\}$ (again, could be incoming or outgoing weights--though not mixed), we calculate the following:

$$ C_{\text{wedge, a}} = \frac{2}{k_a(k_a - 1)}\sum_{i<j} \sqrt{|\hat{a}_i \hat{a}_j|} $$


Where $ \hat{a}_i = \frac{a_i}{ \max_{j} (a_j)}$ is a normalization and $k_a$ is $a$'s incoming or outgoing degree (though not both). 

We calculate this below in the following way:

Let $a$ be an array of weights. $A = a a^T$. Subtract off the diagonal elements of A: $B = A - \text{diag}(A)$. Then we flatten the array, apply the square root to each element, and sum over the elements. We have to divide by 2 to get rid of double counting the elements (we take the sum over all $(i,j)$ and not just when $i < j$. This is purely because I'm lazy, and I'm convinced NumPy is better than "for" loops.




In [1]:
import numpy as np

from keras.models import load_model
from matplotlib import pyplot as plt
%matplotlib inline

# load a test model
model_filename = 'keras_mnist_modelv1.h5'
model = load_model(model_filename)
model.summary()

Using TensorFlow backend.
/Users/nzolman/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 26, 26)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 24, 24)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 12, 12)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 12, 12)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               589952    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

### Wedge Motifs

I allow for the normalization to change in case one wanted more control over whether the normalization was globally calculated (within a layer) or locally calculated (at a node)

In [2]:
# Given an array of weights, calculate the pairwise geometric mean and sum over it


def wedge_cluster(node, norm = 1):
    m = norm # maximum value
    
    node = node/m  # "normalize" the vector

    nodeMat = np.outer(node, node) # Compute the outer product

    nodeMat = nodeMat - np.diag(np.diag(nodeMat)) # subtract off the diagonal

    sqrtArr = np.sqrt(np.abs(nodeMat.flatten())) # flatten and take the square root

    return 2* np.sum(sqrtArr)/(2* node.size * (node.size - 1)) # normalize properly

def wedge_local(weights):
    return np.array([wedge_cluster(node, norm = np.max(np.abs(node))) for node in weights])

def wedge_global(weights):
    m = np.max(np.abs(weights))
    return np.array([wedge_cluster(node, norm = m) for node in weights])

In [ ]:
# pull in the layer and the incoming weights
dense_layer_in= model.layers[-3]
dense128_incoming = np.array(dense_layer_in.get_weights())[0].T

dense_layer_out= model.layers[-1]
dense128_outgoing = np.array(dense_layer_out.get_weights())[0]

In [ ]:
wedge_128_in_local = wedge_local(dense128_incoming)
wedge_128_in_global = wedge_global(dense128_incoming)

wedge_128_outgoing_local = wedge_local(dense128_outgoing)
wedge_128_outgoing_global = wedge_global(dense128_outgoing) 


In [ ]:
plt.title('Incoming Weights')
plt.plot(wedge_128_in_local, label = 'local')
plt.plot(wedge_128_in_global, label = 'global')
plt.legend()
plt.show()

plt.title('Outgoing Weights')
plt.plot(wedge_128_outgoing_local, label = 'local')
plt.plot(wedge_128_outgoing_global, label = 'global')
plt.legend()
plt.show()

plt.title('Incoming')
plt.hist(wedge_128_in_local, 20, alpha = 0.4, label = 'local')
plt.hist(wedge_128_in_global, 20, alpha = 0.4, label = 'global')
plt.legend()
plt.show()

plt.title('Outgoing')
plt.hist(wedge_128_outgoing_local, 20, alpha = 0.4, label = 'local')
plt.hist(wedge_128_outgoing_global, 20, alpha = 0.4, label = 'global')
plt.legend()
plt.show()



### Discussion: 

As you can see, there is certainly a qualitative difference for both the incoming and outgoing weights. If the norms were computed at the node level, we get larger coefficients. And as one would expect, we can get large peaks for local corresponding to similarly valued low-weights on a node.

## <span style="color:red"> 4-cycle clustering </span>

Suppose we have a left layer, $L$ with weights $L_{l,i}$ corresponding from node $l$ in the layer to node $i$ in the middle layer. And suppose we have a right layer $R$ with weights $R_{i,r}$ corresponding to the weight from node $i$ in the middle layer and node $r$ in the right layer. 

So for nodes $i, j$ in the middle layer, we can calculate the four-cycle clustering coefficient corresponding to it. 

$$ C_{\text{4-cylce, i}} = \frac{1}{n_L n_R (n-1)} \sum_{\substack{j \neq i\\l \leq n_L\\r \leq n_R}} \hat{L}_{l,i} \hat{L}_{l,j} \hat{R}_{i,r} \hat{R}_{j,r}$$


Where $\hat{L}_{l,i} = \sqrt[4]{\frac{|L_{l,i}|}{\text{max}_l(|L_{l,i}|)}}$ and likewise for $\hat{R}_{i,r}$.

We can calculate this by calculating $\hat{L}^T \hat{L}$ and $\hat{R} \hat{R}^T $

$$ C_{\text{4-cylce, a}} = \sum_{j \neq i} \left(\hat{L}^T \hat{L})\right)_{ij} \left(\hat{R} \hat{R}^T \right)_{ij}$$

In [ ]:
def quad_coeff(in_weights, out_weights, norm_in = 0, norm_out = 0):

    n_in = in_weights.shape[0]
    n = in_weights.shape[1]
    n_out = out_weights.shape[1]
    motif_poss = (n_in)*(n_out)*(n-1)

    # Default norm is the respective layer norms
    if (norm_out * norm_in == 0):
        norm_in = np.max(np.abs(in_weights))
        norm_out = np.max(np.abs(out_weights))
    
    # normalize the weights
    in_weights = in_weights/norm_in
    out_weights = out_weights/norm_out
    
    # transform entries to be to the 1/4th power
    in_weights = np.power(np.abs(in_weights), .25)
    out_weights = np.power(np.abs(out_weights), .25)
  
    # Caclulate A^t.A and B.B^t
    in_matrix = np.matmul(in_weights.T, in_weights)
    out_matrix = np.matmul(out_weights, out_weights.T)
    
    # subtract off the diagonals
    in_matrix = in_matrix - np.diag(np.diag(in_matrix))
    out_matrix = out_matrix - np.diag(np.diag(out_matrix))
    
    # entry-wise multiplication
    quad_matrix = np.multiply(in_matrix, out_matrix)
    
    # Return normalized coeff
    return(1/(motif_poss) * np.sum(quad_matrix, axis = 0))
    
    

In [ ]:
plt.plot(quad_coeff(dense128_incoming.T, dense128_outgoing))
plt.show()

plt.hist(quad_coeff(dense128_incoming.T, dense128_outgoing), 20, alpha = 0.4, label = 'global')
plt.show()